In [28]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [29]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [30]:
Campaign_Query='''



with Camp_data_TP4 as (

select distinct BACCT_NUM as BAN

from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where CAMP_CREATIVE in ('ALR1_TP4')
and date(Campaign_date)<'2022-01-01' 
and OPENED=1

)

, Camp_data_TP4X as (

select distinct BACCT_NUM as BAN

from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where CAMP_CREATIVE in ('ALR1_TP4X')
and date(Campaign_date)<'2022-01-01' 

)


, Creative_data as (

select distinct BACCT_NUM as BAN, 1 as creative_flag
from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where REGEXP_CONTAINS (CAMP_CREATIVE, r'ALR1_TP5')
-- and  date(Campaign_date)>='2022-01-01' and date(Campaign_date)<='2022-06-30'

)


,merged_data as (

select * from Camp_data_TP4

UNION ALL

select * from Camp_data_TP4X

)



, Final_data as (

select distinct a.BAN, b.creative_flag 
from merged_data a
left join Creative_data b
on a.BAN=b.BAN

)
select distinct BAN from  Final_data
where creative_flag is Null


'''

In [31]:
Camp_DF=extract_bq_data(bq_client, sql=Campaign_Query)

In [32]:
Camp_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   BAN     62 non-null     Int64
dtypes: Int64(1)
memory usage: 686.0 bytes


In [33]:
# pd.DataFrame(Camp_DF.groupby(['CAMP_CREATIVE','OPENED']).agg(
    
#     Customer_count= ('BACCT_NUM','nunique'),
#     # Email_Opened_count=('OPENED','sum'),
#     # CT_Opened_count=('CLICKTHROUGH','sum'),
#     # softbounce_count=('SOFTBOUNCE','sum'),
#     # Hardbounce_count=('HARDBOUNCE','sum'),
#     # Unsub_count=('UNSUBSCRIBE','sum'),
#     campaign_date_min=('Campaign_date','min'),
#     campaign_date_max=('Campaign_date','max')

    
#     # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [34]:
# def Camp_creative_making(row):
    
#     if row['CAMP_CREATIVE']=='ALR1_TP5X_V1' or row['CAMP_CREATIVE']=='ALR1_TP5_V1':
#         return 'V1'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V2' or row['CAMP_CREATIVE']=='ALR1_TP5_V2':
#         return 'V2'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V3' or row['CAMP_CREATIVE']=='ALR1_TP5_V3':
#         return 'V3'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V4' or row['CAMP_CREATIVE']=='ALR1_TP5_V4':
#         return 'V4'
#     else:
#         return "None"

In [35]:
# Camp_DF['Campaign_date'].value_counts().sort_index()

In [36]:
# Camp_DF['CAMP_CREATIVE_New']=Camp_DF.apply(Camp_creative_making,axis=1)

In [37]:
# Camp_DF['CAMP_CREATIVE_New'].value_counts().sort_index()

In [38]:
# pd.DataFrame(Camp_DF.groupby(['CAMP_CREATIVE_New','OPENED']).agg(
    
#     Customer_count= ('BACCT_NUM','nunique'),
#     # Email_Opened_count=('OPENED','sum'),
#     # CT_Opened_count=('CLICKTHROUGH','sum'),
#     # softbounce_count=('SOFTBOUNCE','sum'),
#     # Hardbounce_count=('HARDBOUNCE','sum'),
#     # Unsub_count=('UNSUBSCRIBE','sum'),
#     campaign_date_min=('Campaign_date','min'),
#     campaign_date_max=('Campaign_date','max')

    
#     # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [39]:
config= bigquery.job.LoadJobConfig()
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved'

bq_table_instance= bq_client.load_table_from_dataframe(Camp_DF, Table_BQ,job_config=config)

In [40]:
H2_2022_Snapshot='''




DECLARE target_date_start_H2 DATE DEFAULT "2022-07-01";
DECLARE target_date_start DATE DEFAULT "2022-12-01";
DECLARE target_date_end DATE DEFAULT "2022-12-31";
DECLARE target_date_start_next_month DATE DEFAULT "2023-01-01";



With Camp_V1_BAN as (

select distinct BAN from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved`

)


, prod_snpsht_SMHM_start_H2 as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH_start_H2
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_start_H2


)

, prod_snpsht_SMHM_end_H2 as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH_end_H2
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_end


)
, ARPU_Calculation_start_H1 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2021-12-01'
  and DATE(bill.bill_dt)< '2022-01-01'
  group by ban,bill_year_month
)


, ARPU_Calculation_start_H2 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2022-06-01'
  and DATE(bill.bill_dt)< '2022-07-01'
  group by ban,bill_year_month
)

, ARPU_Calculation_end_H2 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2022-12-01'
  and DATE(bill.bill_dt)< '2023-01-01'
  group by ban,bill_year_month
)

select a.BAN
,b.count_of_BAN_FFH_start_H2
,d.count_of_BAN_FFH_end_H2
,c.bill_year_month as bill_year_month_H1_2022_start
,c.ARPU_SMHM as ARPU_SMHM_H1_2022_start
,c.ARPU_FFH as ARPU_FFH_H1_2022_start
,e.bill_year_month as bill_year_month_H2_2022_start
,e.ARPU_SMHM as ARPU_SMHM_H2_2022_start
,e.ARPU_FFH as ARPU_FFH_H2_2022_start
,f.bill_year_month as bill_year_month_H2_2022_end
,f.ARPU_SMHM as ARPU_SMHM_H2_2022_end
,f.ARPU_FFH as ARPU_FFH_H2_2022_end

 from Camp_V1_BAN a
left join prod_snpsht_SMHM_start_H2 b
on a.BAN=b.BAN
left join prod_snpsht_SMHM_end_H2 d
on a.BAN=d.BAN
left join ARPU_Calculation_start_H1 c
on a.BAN=c.ban
left join ARPU_Calculation_start_H2 e
on a.BAN=e.ban
left join ARPU_Calculation_end_H2 f
on a.BAN=f.ban


'''

In [41]:
H2_2022_Snapshot_count=extract_bq_data(bq_client, sql=H2_2022_Snapshot)

In [42]:
H2_2022_Snapshot_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   BAN                            62 non-null     Int64 
 1   count_of_BAN_FFH_start_H2      53 non-null     Int64 
 2   count_of_BAN_FFH_end_H2        48 non-null     Int64 
 3   bill_year_month_H1_2022_start  62 non-null     object
 4   ARPU_SMHM_H1_2022_start        61 non-null     object
 5   ARPU_FFH_H1_2022_start         62 non-null     object
 6   bill_year_month_H2_2022_start  62 non-null     object
 7   ARPU_SMHM_H2_2022_start        58 non-null     object
 8   ARPU_FFH_H2_2022_start         62 non-null     object
 9   bill_year_month_H2_2022_end    62 non-null     object
 10  ARPU_SMHM_H2_2022_end          53 non-null     object
 11  ARPU_FFH_H2_2022_end           62 non-null     object
dtypes: Int64(3), object(9)
memory usage: 6.1+ KB


In [43]:
H2_2022_Snapshot_count['Place_holder']='Place_holder'

In [44]:
H2_2022_Snapshot_count.head()

,BAN,count_of_BAN_FFH_start_H2,count_of_BAN_FFH_end_H2,bill_year_month_H1_2022_start,ARPU_SMHM_H1_2022_start,ARPU_FFH_H1_2022_start,bill_year_month_H2_2022_start,ARPU_SMHM_H2_2022_start,ARPU_FFH_H2_2022_start,bill_year_month_H2_2022_end,ARPU_SMHM_H2_2022_end,ARPU_FFH_H2_2022_end,Place_holder
0,603578943,1,1,2021-12,45.000000000,50.400000000,2022-06,45.000000000,50.400000000,2022-12,45.000000000,50.400000000,Place_holder
1,212490756,1,1,2021-12,60.000000000,290.350000000,2022-06,60.000000000,310.160000000,2022-12,67.000000000,233.450000000,Place_holder
2,602050313,1,<NA>,2021-12,56.410000000,106.860000000,2022-06,214.770000000,143.780000000,2022-12,-5.210000000,104.000000000,Place_holder
3,603566897,<NA>,<NA>,2021-12,-13.500000000,-15.130000000,2022-06,None,0E-9,2022-12,None,0E-9,Place_holder
4,124265235,1,1,2021-12,43.340000000,91.000000000,2022-06,80.830000000,176.000000000,2022-12,77.000000000,118.000000000,Place_holder


In [45]:
pd.DataFrame(H2_2022_Snapshot_count.groupby(['Place_holder']).agg(
    
    Customer_count= ('BAN','nunique'),
    Customer_count_1= ('count_of_BAN_FFH_start_H2','sum'),
    Customer_count_2= ('count_of_BAN_FFH_end_H2','sum'),
    ARPU_SMHM_H1_2022_start=('ARPU_SMHM_H1_2022_start','mean'),
    ARPU_SMHM_H2_2022_start=('ARPU_SMHM_H2_2022_start','mean'),
    ARPU_SMHM_H2_2022_end=('ARPU_SMHM_H2_2022_end','mean'),
    ARPU_FFH_H1_2022_start=('ARPU_FFH_H1_2022_start','mean'),
    ARPU_FFH_H2_2022_start=('ARPU_FFH_H2_2022_start','mean'),
    ARPU_FFH_H2_2022_end=('ARPU_FFH_H2_2022_end','mean'),
    # Email_Opened_count=('OPENED','sum'),
    # CT_Opened_count=('CLICKTHROUGH','sum'),
    # softbounce_count=('SOFTBOUNCE','sum'),
    # Hardbounce_count=('HARDBOUNCE','sum'),
    # Unsub_count=('UNSUBSCRIBE','sum'),
    # campaign_date_min=('Campaign_date','min'),
    # campaign_date_max=('Campaign_date','max')

    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Place_holder,Customer_count,Customer_count_1,Customer_count_2,ARPU_SMHM_H1_2022_start,ARPU_SMHM_H2_2022_start,ARPU_SMHM_H2_2022_end,ARPU_FFH_H1_2022_start,ARPU_FFH_H2_2022_start,ARPU_FFH_H2_2022_end
0,Place_holder,62,53,48,52.786721,53.964138,59.898302,102.774839,99.602903,92.693387


In [18]:
H2_2022_Snapshot_count['BAN'].value_counts()

212490756    1
602050313    1
603754961    1
603586385    1
603596425    1
602694930    1
603582022    1
602062216    1
603591416    1
601929358    1
124265235    1
603646192    1
603581864    1
603625227    1
603587021    1
603566897    1
218840580    1
603655379    1
603604685    1
603075527    1
229458660    1
603571958    1
601078382    1
204752434    1
225788162    1
603242639    1
603674713    1
603570048    1
603207878    1
603654639    1
602986094    1
603602390    1
603591130    1
603701182    1
226022951    1
603716441    1
603612643    1
603753286    1
603426976    1
201685346    1
600904093    1
603593253    1
603578153    1
224251052    1
201782324    1
300053070    1
603603293    1
603645788    1
603578943    1
601643369    1
603652328    1
603573470    1
123811482    1
603582763    1
603583711    1
603260125    1
603561700    1
603590747    1
603558708    1
603568481    1
232297248    1
603603407    1
Name: BAN, dtype: Int64

In [58]:
H2_2022_Snapshot_count[H2_2022_Snapshot_count['BAN']==208624864]

,BAN,count_of_BAN_FFH_start_H2,count_of_BAN_FFH_end_H2,bill_year_month_H1_2022_start,ARPU_SMHM_H1_2022_start,ARPU_FFH_H1_2022_start,bill_year_month_H2_2022_start,ARPU_SMHM_H2_2022_start,ARPU_FFH_H2_2022_start,bill_year_month_H2_2022_end,ARPU_SMHM_H2_2022_end,ARPU_FFH_H2_2022_end


In [59]:
# H2_2022_Snapshot_count.to_csv('ALR_Campaign_Control_2022H2.csv',index=False)

In [58]:
Contract_status_Query='''






with Intial_snapshot as (
        
     SELECT 
     distinct b.BAN as ban_start_H1
     ,a.pi_prod_instnc_stat_cd as status_intial_start_H1
    , max(pi_cntrct_end_ts) as latest_contract_end_dt_start_h1
    ,pi_prod_instnc_resrc_str as res_id_intial

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-01-01'
    and a.pi_prod_instnc_typ_cd='SMHM'
    and a.pi_prod_instnc_stat_cd='A'

    GROUP BY 
     b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)


, H2_start_snapshot as (
        
        SELECT
     distinct b.BAN as ban_start_h2
    ,a.pi_prod_instnc_stat_cd as status_start_h2
    , max(pi_cntrct_end_ts) as contract_end_dt_start_h2
     ,pi_prod_instnc_resrc_str as res_id_start_h2
    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-07-01'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY 
      b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)

, H2_end_snapshot as (
        
        SELECT
     distinct b.BAN as ban_end_h2
    ,a.pi_prod_instnc_stat_cd as status_end_h2
    , max(pi_cntrct_end_ts) as contract_end_dt_end_h2
    ,pi_prod_instnc_resrc_str as res_id_end_h2

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-12-31'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY 
     b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)




select 
distinct a.ban_start_H1,a.status_intial_start_H1,a.latest_contract_end_dt_start_h1,b.*,c.*
from Intial_snapshot a
left JOIN  H2_start_snapshot b
on a.ban_start_H1=b.ban_start_h2 and a.res_id_intial=b.res_id_start_h2
left join H2_end_snapshot c
on a.ban_start_H1=c.ban_end_h2 and a.res_id_intial=c.res_id_end_h2

'''

In [59]:
Contract_DF=extract_bq_data(bq_client, sql=Contract_status_Query)

In [60]:
Contract_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ban_start_H1                     57 non-null     Int64 
 1   status_intial_start_H1           57 non-null     object
 2   latest_contract_end_dt_start_h1  57 non-null     object
 3   ban_start_h2                     57 non-null     Int64 
 4   status_start_h2                  57 non-null     object
 5   contract_end_dt_start_h2         57 non-null     object
 6   res_id_start_h2                  57 non-null     object
 7   ban_end_h2                       57 non-null     Int64 
 8   status_end_h2                    57 non-null     object
 9   contract_end_dt_end_h2           57 non-null     object
 10  res_id_end_h2                    57 non-null     object
dtypes: Int64(3), object(8)
memory usage: 5.2+ KB


In [61]:
Contract_DF.head()

,ban_start_H1,status_intial_start_H1,latest_contract_end_dt_start_h1,ban_start_h2,status_start_h2,contract_end_dt_start_h2,res_id_start_h2,ban_end_h2,status_end_h2,contract_end_dt_end_h2,res_id_end_h2
0,603582763,A,9999-12-31 00:00:00,603582763,C,9999-12-31 00:00:00,1002225475,603582763,C,9999-12-31 00:00:00,1002225475
1,600904093,A,2021-10-17 00:00:00,600904093,A,2021-10-17 00:00:00,1002387853,600904093,A,2021-10-17 00:00:00,1002387853
2,603260125,A,2021-11-27 00:00:00,603260125,A,2021-11-27 00:00:00,1002850380,603260125,C,2021-11-27 00:00:00,1002850380
3,201782324,A,2026-11-19 00:00:00,201782324,A,2026-11-19 00:00:00,1011795854,201782324,A,2026-11-19 00:00:00,1011795854
4,603652328,A,2021-09-12 00:00:00,603652328,A,2021-09-12 00:00:00,1002352888,603652328,A,2021-09-12 00:00:00,1002352888


In [62]:
Contract_DF['ban_start_H1'].value_counts()

603582763    1
600904093    1
603260125    1
201782324    1
603652328    1
603578153    1
603646192    1
603602390    1
603582022    1
603558708    1
601078382    1
602050313    1
603593253    1
603625227    1
603754961    1
603701182    1
212490756    1
226022951    1
603590747    1
603242639    1
602062216    1
224251052    1
603586385    1
601643369    1
204752434    1
603603293    1
603581864    1
603654639    1
300053070    1
123811482    1
603645788    1
603612643    1
603583711    1
603591130    1
229458660    1
603207878    1
603753286    1
124265235    1
601929358    1
603596425    1
232297248    1
603603407    1
603591416    1
603561700    1
603571958    1
603604685    1
201685346    1
603568481    1
603716441    1
603573470    1
218840580    1
603578943    1
603570048    1
602694930    1
225788162    1
603587021    1
603674713    1
Name: ban_start_H1, dtype: Int64

In [65]:
Contract_DF[Contract_DF.ban_start_H1==601929358]

,ban_start_H1,status_intial_start_H1,latest_contract_end_dt_start_h1,ban_start_h2,status_start_h2,contract_end_dt_start_h2,res_id_start_h2,ban_end_h2,status_end_h2,contract_end_dt_end_h2,res_id_end_h2
38,601929358,A,2024-11-21 00:00:00,601929358,A,2024-11-21 00:00:00,1012228215,601929358,A,2024-11-21 00:00:00,1012228215


In [64]:
Contract_DF.to_csv('ALR_Campaign_Control_eligible_not_received_status_2022H2.csv',index=False)